In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_selection.relevance import calculate_relevance_table

In [15]:
train_weather = pd.read_csv('/Users/rkhu/Desktop/upenn/other/maize_gxe/data/Training_Data/4_Training_Weather_Data_2014_2021.csv')
train_trait = pd.read_csv('/Users/rkhu/Desktop/upenn/other/maize_gxe/data/Training_Data/1_Training_Trait_Data_2014_2021.csv')

train_weather['Date'] = pd.to_datetime(train_weather['Date'], format='%Y%m%d')
train_weather['Month_Day'] = train_weather['Date'].dt.strftime('%m-%d')
train_weather['Field_Location'] = train_weather['Env'].str.replace('_\d{4}', '')
train_weather['Train_Test'] = 'Train'

/var/folders/x1/68s4ss8j44bgn7dv42vcp9qh0000gp/T/ipykernel_40888/3576293083.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  train_weather['Field_Location'] = train_weather['Env'].str.replace('_\d{4}', '')


In [16]:
test_weather = pd.read_csv('/Users/rkhu/Desktop/upenn/other/maize_gxe/data/Testing_Data/4_Testing_Weather_Data_2022.csv')

test_weather['Date'] = pd.to_datetime(test_weather['Date'], format='%Y%m%d')
test_weather['Month_Day'] = test_weather['Date'].dt.strftime('%m-%d')
test_weather['Field_Location'] = test_weather['Env'].str.replace('_\d{4}', '')
test_weather['Train_Test'] = 'Test'

/var/folders/x1/68s4ss8j44bgn7dv42vcp9qh0000gp/T/ipykernel_40888/3934614273.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  test_weather['Field_Location'] = test_weather['Env'].str.replace('_\d{4}', '')


In [ ]:
len(pd.Series(list(train_weather['Env'].unique()) + list(test_weather['Env'].unique())).unique())

238

In [26]:
len(train_test_weather['Env'].unique())

238

In [4]:
train_test_weather = pd.concat([train_weather, test_weather])
train_test_weather.index = train_test_weather['Field_Location'] + '_' + train_test_weather['Date'].astype('str')

In [27]:
train_test_weather.head()

,Env,Date,QV2M,T2MDEW,PS,RH2M,WS2M,GWETTOP,ALLSKY_SFC_SW_DWN,ALLSKY_SFC_PAR_TOT,...,T2M_MIN,T2MWET,GWETROOT,T2M,GWETPROF,ALLSKY_SFC_SW_DNI,PRECTOTCORR,Month_Day,Field_Location,Train_Test
ARH1_2016-01-01,ARH1_2016,2016-01-01,3.54,-0.78,102.34,77.00,2.15,0.84,8.21,41.96,...,-0.70,1.15,0.83,3.08,0.8,5.96,0.0,01-01,ARH1,Train
ARH1_2016-01-02,ARH1_2016,2016-01-02,3.23,-1.91,102.04,74.62,1.49,0.84,11.28,55.13,...,-3.10,0.42,0.83,2.74,0.8,16.13,0.0,01-02,ARH1,Train
ARH1_2016-01-03,ARH1_2016,2016-01-03,4.09,1.05,101.59,80.69,1.95,0.84,9.78,49.21,...,-1.29,2.72,0.83,4.38,0.8,18.36,0.0,01-03,ARH1,Train
ARH1_2016-01-04,ARH1_2016,2016-01-04,2.87,-3.49,102.24,79.88,3.45,0.84,7.35,35.66,...,-4.00,-1.79,0.83,-0.09,0.8,10.87,0.0,01-04,ARH1,Train
ARH1_2016-01-05,ARH1_2016,2016-01-05,2.81,-3.64,102.37,78.81,1.95,0.84,13.00,62.04,...,-4.59,-1.74,0.82,0.16,0.8,27.02,0.0,01-05,ARH1,Train


In [29]:
train_test_weather.shape

(85595, 21)

In [30]:
X = train_test_weather.drop(['Env','Date','Month_Day','Field_Location','Train_Test'], axis=1)
X_train = X.loc[train_test_weather.loc[train_test_weather['Train_Test']=='Train'].index]

In [32]:
%%time
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler = min_max_scaler.fit(X_train)
X_scaled = pd.DataFrame(min_max_scaler.transform(X), index=X.index, columns=X.columns)

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5, weights="uniform")
imputer.fit(X_scaled)
X_imputed = pd.DataFrame(imputer.transform(X_scaled), index=X_scaled.index, columns=X_scaled.columns)

CPU times: user 1min 8s, sys: 35.9 s, total: 1min 44s
Wall time: 1min 17s


In [33]:
train_test_weather_imputed = pd.merge(train_test_weather.loc[:,['Env','Date','Month_Day','Field_Location','Train_Test']], 
                                      X_imputed, left_index=True, right_index=True)

In [36]:
len(train_test_weather_imputed['Env'].unique())

238

In [53]:
%%time
train_test_weather_extracted_features = extract_features(train_test_weather_imputed.drop(['Date','Field_Location','Train_Test'], axis=1), 
                                                         column_id="Env", 
                                                         column_sort="Month_Day")

Feature Extraction: 100%|███████████████████████████| 25/25 [05:26<00:00, 13.06s/it]


CPU times: user 3.4 s, sys: 710 ms, total: 4.11 s
Wall time: 5min 28s


In [54]:
train_trait = pd.read_csv('/Users/rkhu/Desktop/upenn/other/maize_gxe/data/Training_Data/1_Training_Trait_Data_2014_2021.csv')
train_env_yield = train_trait.loc[train_trait['Env'].isin(train_test_weather_extracted_features.index),:].groupby('Env')['Yield_Mg_ha'].mean()

In [55]:
features_relevance_table = calculate_relevance_table(train_test_weather_extracted_features.loc[train_env_yield.index].dropna(axis=1), 
                                                     train_env_yield)

In [56]:
%%time
train_test_weather_extracted_features_corr = train_test_weather_extracted_features.corr().abs()

CPU times: user 59.3 s, sys: 1.02 s, total: 1min
Wall time: 1min


In [57]:
%%time
corr_thresh = 0.8
i = 0
filtered_features_relevance_table = features_relevance_table
while i < filtered_features_relevance_table.shape[0]:
    feature = filtered_features_relevance_table['feature'][i]
    corr_features = train_test_weather_extracted_features_corr.index[train_test_weather_extracted_features_corr.loc[feature] > corr_thresh].drop(feature, errors='ignore')
    corr_features = corr_features[corr_features.isin(filtered_features_relevance_table.index)]
    filtered_features_relevance_table = filtered_features_relevance_table.drop(corr_features, axis=0)
    i = i + 1

CPU times: user 8.45 s, sys: 72.2 ms, total: 8.52 s
Wall time: 8.51 s


In [58]:
# Select features based on mean yield per environment
train_test_weather_filtered_features = select_features(train_test_weather_extracted_features.loc[train_env_yield.index].dropna(axis=1)[filtered_features_relevance_table.index], 
                                                       train_env_yield, fdr_level=0.001)

In [68]:
final_weather_extracted_features = train_test_weather_extracted_features[train_test_weather_filtered_features.columns]

In [69]:
final_weather_extracted_features.to_csv('4_Training_Testing_Weather_Data_tsfresh_extracted_features.csv')
# final_weather_extracted_features = pd.read_csv('4_Training_Testing_Weather_Data_tsfresh_extracted_features.csv', index_col=0)

In [20]:
na_count = train_test_weather.groupby('Env').count().rsub(train_test_weather.groupby('Env').size(), axis=0)
na_count['total_na_count'] = na_count.sum(axis=1)

In [28]:
na_count

,Date,QV2M,T2MDEW,PS,RH2M,WS2M,GWETTOP,ALLSKY_SFC_SW_DWN,ALLSKY_SFC_PAR_TOT,T2M_MAX,T2M_MIN,T2MWET,GWETROOT,T2M,GWETPROF,ALLSKY_SFC_SW_DNI,PRECTOTCORR,Month_Day,Field_Location,total_na_count
Env,,,,,,,,,,,,,,,,,,,,
ARH1_2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ARH1_2017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ARH1_2018,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ARH2_2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ARH2_2017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WIH2_2021,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
WIH2_2022,0,5,5,5,5,5,254,10,254,5,5,5,254,5,254,254,5,0,0,1330
WIH3_2020,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
na_count.to_csv('weather_feature_na_count_by_env.csv')